<a href="https://colab.research.google.com/github/PsorTheDoctor/artificial-intelligence/blob/master/modern_approach/forward_forward/forward_forward_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Forward-Forward propagation

In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.optim import Adam
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda
from torch.utils.data import DataLoader

In [3]:
def MNIST_loaders(train_batch_size=50000, test_batch_size=10000):
  transform = Compose([
      ToTensor(),
      Normalize((0.1307,), (0.3081,)),
      Lambda(lambda x: torch.flatten(x))
  ])
  train_loader = DataLoader(
      MNIST('./data/', train=True, download=True, transform=transform),
      batch_size=train_batch_size, shuffle=True
  )
  test_loader = DataLoader(
      MNIST('./data/', train=False, download=True, transform=transform),
      batch_size=test_batch_size, shuffle=False
  )
  return train_loader, test_loader

def overlay_y_on_x(x, y):
  x_ = x.clone()
  x_[:, :10] *= 0.0
  x_[range(x.shape[0]), y] = x.max()
  return x_

In [4]:
class Net(torch.nn.Module):
  def __init__(self, dims):
    super().__init__()
    self.layers = []
    for d in range(len(dims) - 1):
      self.layers += [Layer(dims[d], dims[d + 1]).cuda()]

  def predict(self, x):
    goodness_per_label = []
    for label in range(10):
      h = overlay_y_on_x(x, label)
      goodness = []
      for layer in self.layers:
        h = layer(h)
        goodness += [h.pow(2).mean(1)]
      goodness_per_label += [sum(goodness).unsqueeze(1)]
    goodness_per_label = torch.cat(goodness_per_label, 1)
    return goodness_per_label.argmax(1)

  def train(self, x_pos, x_neg):
    h_pos, h_neg = x_pos, x_neg
    for i, layer in enumerate(self.layers):
      print('Training layer', i, '...')
      h_pos, h_neg = layer.train(h_pos, h_neg)

In [5]:
class Layer(nn.Linear):
  def __init__(self, in_features, out_features, bias=True, device=None, dtype=None):
    super().__init__(in_features, out_features, bias, device, dtype)
    self.relu = torch.nn.ReLU()
    self.opt = Adam(self.parameters(), lr=0.03)
    self.threshold = 2.0
    self.epochs = 1000

  def forward(self, x):
    x_direction = x / (x.norm(2, 1, keepdim=True) + 1e-4)
    return self.relu(
        torch.mm(x_direction, self.weight.T) +
        self.bias.unsqueeze(0)
    )

  def train(self, x_pos, x_neg):
    for i in tqdm(range(self.epochs)):
      g_pos = self.forward(x_pos).pow(2).mean(1)
      g_neg = self.forward(x_neg).pow(2).mean(1)
      loss = torch.log(1 + torch.exp(torch.cat([
          -g_pos + self.threshold,
          g_neg - self.threshold
      ]))).mean()
      self.opt.zero_grad()
      loss.backward()
      self.opt.step()
    return self.forward(x_pos).detach(), self.forward(x_neg).detach()

In [7]:
torch.manual_seed(42)
train_loader, test_loader = MNIST_loaders()

net = Net([784, 500, 500])
x, y = next(iter(train_loader))
x, y = x.cuda(), y.cuda()
x_pos = overlay_y_on_x(x, y)
rnd = torch.randperm(x.size(0))
x_neg = overlay_y_on_x(x, y[rnd])
net.train(x_pos, x_neg)

print('Train error:', 1.0 - net.predict(x).eq(y).float().mean().item())

x_test, y_test = next(iter(test_loader))
x_test, y_test = x_test.cuda(), y_test.cuda()

print('Test error:', 1.0 - net.predict(x_test).eq(y_test).float().mean().item())

Training layer 0 ...


100%|██████████| 1000/1000 [00:59<00:00, 16.67it/s]


Training layer 1 ...


100%|██████████| 1000/1000 [00:39<00:00, 25.17it/s]


Train error: 0.07084000110626221
Test error: 0.06929999589920044


Credits: Mohammad Pezeshki (https://github.com/mohammadpz)